# Recommend Items Frequently Purchased Together
This notebook demonstrates how recommending items that are frequently purchased together is effective. The current best scoring public notebook [here][1] recommends to customers those customers' last purchases and scores public LB 0.020. In this notebook here, we will begin with that idea and add recommending items that are frequently purchased together with a customers' previous purchaes. This notebook improves the LB and scores LB 0.021. This notebook's strategy is as follows:
* recommend items previously purchased [idea here][1]
* recommend items that are bought together with previous purchases [idea here][2]
* recommend popular items [idea here][1]

[1]: https://www.kaggle.com/hengzheng/time-is-our-best-friend-v2
[2]: https://www.kaggle.com/cdeotte/customers-who-bought-this-frequently-buy-this

# RAPIDS cuDF
We will use RAPIDS cuDF for fast dataframe operations

In [1]:
import cudf
print('RAPIDS version',cudf.__version__)

RAPIDS version 21.10.01


In [2]:
log_date = '2020-09-23'
Ntop = 30
i2i_Ntop = 2
#ch = 'off'
ch = 'on'

# Load Transactions, Reduce Memory
Discussion about reducing memory is [here][1]

[1]: https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/308635

In [3]:
train = cudf.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
train['customer_id'] = train['customer_id'].str[-16:].str.hex_to_int().astype('int64')
train['article_id'] = train.article_id.astype('int32')
train.t_dat = cudf.to_datetime(train.t_dat)
train = train[['t_dat','customer_id','article_id']]
train = train.loc[train['t_dat'] < cudf.to_datetime(log_date)]
train.to_parquet('train.pqt',index=False)
print( train.shape )
train.head()

(31788324, 3)


,t_dat,customer_id,article_id
0,2018-09-20,-6846340800584936,663713001
1,2018-09-20,-6846340800584936,541518023
2,2018-09-20,-8334631767138808638,505221004
3,2018-09-20,-8334631767138808638,685687003
4,2018-09-20,-8334631767138808638,685687004


# Find Each Customer's Last Week of Purchases
Our final predictions will have the row order from of our dataframe. Each row of our dataframe will be a prediction. We will create the `predictionstring` later by `train.groupby('customer_id').article_id.sum()`. Since `article_id` is a string, when we groupby sum, it will concatenate all the customer predictions into a single string. It will also create the string in the order of the dataframe. So as we proceed in this notebook, we will order the dataframe how we want our predictions ordered.

In [4]:
tmp = train.groupby('customer_id').t_dat.max().reset_index()
tmp.columns = ['customer_id','max_dat']
train = train.merge(tmp,on=['customer_id'],how='left')
train['diff_dat'] = (train.max_dat - train.t_dat).dt.days
train = train.loc[train['diff_dat']<=13]
print('Train shape:',train.shape)

Train shape: (5609905, 5)


# (1) Recommend Most Often Previously Purchased Items
Note that many operations in cuDF will shuffle the order of the dataframe rows. Therefore we need to sort afterward because we want the most often previously purchased items first. Because this will be the order of our predictons. Since we sort by `ct` and then `t_dat` will will recommend items that have been purchased more frequently first followed by items purchased more recently second.

In [5]:
tmp = train.groupby(['customer_id','article_id'])['t_dat'].agg('count').reset_index()
tmp.columns = ['customer_id','article_id','ct']
train = train.merge(tmp,on=['customer_id','article_id'],how='left')
train = train.sort_values(['ct','t_dat'],ascending=False)
train = train.drop_duplicates(['customer_id','article_id'])
train = train.sort_values(['ct','t_dat'],ascending=False)
train.head()

,t_dat,customer_id,article_id,max_dat,diff_dat,ct
1193601,2019-07-16,2729025827381139556,719348003,2019-07-16,0,100
57664,2018-10-04,4485518665254175540,557247001,2018-10-04,0,86
2235052,2020-03-06,-906958334866810496,852521001,2020-03-06,0,81
3600672,2020-07-06,3601599666106972342,685813001,2020-07-06,0,80
886177,2019-05-14,-4601407992705575197,695545001,2019-05-14,0,80


# (2) Recommend Items Purchased Together
In my notebook [here][1], we compute a dictionary of items frequently purchased together. We will load and use that dictionary below. Note that we use the command `drop_duplicates` so that we don't recommend an item that the user has already bought and we have already recommended above. We will need to use Pandas for some commands because RAPIDS cuDF doesn't have two conveinent commands, (1) create new column from dictionary map of another column (2) groupby aggregate strings sum.

We concatenate these rows after the rows containing customers' previous purchases. Therefore we will recommend previous items first and then items purchased together second. Note the trick to convert a column of int32 into a prediction string (using groupby agg str sum) is from notebook [here][2]

[1]: https://www.kaggle.com/cdeotte/customers-who-bought-this-frequently-buy-this
[2]: https://www.kaggle.com/hiroshisakiyama/recommending-items-recently-bought

In [6]:
# USE PANDAS TO MAP COLUMN WITH DICTIONARY
import pandas as pd, numpy as np
train = train.to_pandas()
#pairs = np.load('../input/hmitempairs/pairs_cudf.npy',allow_pickle=True).item()
#pairs = np.load('../input/pairs-cudfnpy/pairs_cudf.npy',allow_pickle=True).item()
#pairs = np.load('../input/pairscudfnpy/pairs_cudf_2020-09-23_2.npy',allow_pickle=True).item()
#pairs = np.load(f'../input/pariscudf0916/pairs_cudf_{log_date}_{i2i_Ntop}.npy',allow_pickle=True).item()

#pairs = np.load(f'../input/pairs-cudf-off-0916/pairs_cudf_2020-09-16_2_off.npy',allow_pickle=True).item()
#pairs = np.load(f'../input/pairs-cudf-on-0916/pairs_cudf_2020-09-16_2_on.npy',allow_pickle=True).item()
#pairs = np.load(f'../input/pairscudfon0909/pairs_cudf_2020-09-09_2_on.npy',allow_pickle=True).item()
pairs = np.load(f'../input/pairscudfon0923/pairs_cudf_2020-09-23_2_on.npy',allow_pickle=True).item()


#pairs = np.load(f'../input/pairscudf0909/pairs_cudf_{log_date}_{i2i_Ntop}.npy',allow_pickle=True).item()

In [7]:
# top N件までのitem pairsを作る
pairs_list =[dict() for i in range(i2i_Ntop)] 
for k, v in pairs.items():
    for i in range(len(v)):
        pairs_list[i][k] = v[i]

In [8]:
train['article_id2'] = train.article_id.map(pairs_list[0]) # 合わせ買い1位
train['article_id3'] = train.article_id.map(pairs_list[1]) # 合わせ買い2位
#train['article_id4'] = train.article_id.map(pairs_list[2]) # 合わせ買い3位

In [9]:
# RECOMMENDATION OF PAIRED ITEMS
train2 = train[['customer_id','article_id2']].copy()
train2 = train2.loc[train2.article_id2.notnull()]
train2 = train2.drop_duplicates(['customer_id','article_id2'])
train2 = train2.rename({'article_id2':'article_id'},axis=1)

train3 = train[['customer_id','article_id3']].copy()
train3 = train3.loc[train3.article_id3.notnull()]
train3 = train3.drop_duplicates(['customer_id','article_id3'])
train3 = train3.rename({'article_id3':'article_id'},axis=1)

#train4 = train[['customer_id','article_id4']].copy()
#train4 = train4.loc[train4.article_id4.notnull()]
#train4 = train4.drop_duplicates(['customer_id','article_id4'])
#train4 = train4.rename({'article_id4':'article_id'},axis=1)

In [10]:
# CONCATENATE PAIRED ITEM RECOMMENDATION AFTER PREVIOUS PURCHASED RECOMMENDATIONS
train = train[['customer_id','article_id']]
train = pd.concat([train,train2,train3],axis=0,ignore_index=True)
train.article_id = train.article_id.astype('int32')
train = train.drop_duplicates(['customer_id','article_id'])

In [11]:
# CONVERT RECOMMENDATIONS INTO SINGLE STRING
train.article_id = ' 0' + train.article_id.astype('str')
preds = cudf.DataFrame( train.groupby('customer_id').article_id.sum().reset_index() )
preds.columns = ['customer_id','prediction']
preds.head()

,customer_id,prediction
0,-9223352921020755230,0673396002 0812167004 0706016001 0812167002 0...
1,-9223343869995384291,0908292002 0910601003 0865929007 0903926002 0...
2,-9223321797620987725,0580600006 0610776035 0610776002 0610776010
3,-9223319430705797669,0470985003 0504155001 0554477005 0562245001 0...
4,-9223308614576639426,0750423005 0750423001 0610776002


# (3) Recommend Last Week's Most Popular Items
After recommending previous purchases and items purchased together we will then recommend the 12 most popular items. Therefore if our previous recommendations did not fill up a customer's 12 recommendations, then it will be filled by popular items.

In [12]:
#train = cudf.read_parquet('train.pqt')
#train.t_dat = cudf.to_datetime(train.t_dat)
#train = train.loc[train.t_dat >= cudf.to_datetime('2020-09-16')]
#top12 = ' 0' + ' 0'.join(train.article_id.value_counts().to_pandas().index.astype('str')[:12])
#print("Last week's top 12 popular items:")
#print( top12 )

# Write Submission CSV
We will merge our predictions onto `sample_submission.csv` and submit to Kaggle.

In [13]:
sub = cudf.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')
sub = sub[['customer_id']]
sub['customer_id_2'] = sub['customer_id'].str[-16:].str.hex_to_int().astype('int64')
sub = sub.merge(preds.rename({'customer_id':'customer_id_2'},axis=1),\
    on='customer_id_2', how='left').fillna('')
del sub['customer_id_2']
#sub.prediction = sub.prediction + top12
sub.prediction = sub.prediction.str.strip()
sub.prediction = sub.prediction.str[:(10*Ntop + (Ntop-1))]
#sub.to_csv(f'submission_{log_date}_{Ntop}.csv',index=False)
sub.to_csv(f'submission_{log_date}_{Ntop}_{ch}.csv',index=False)

sub.head()

,customer_id,prediction
0,060f8aec2fdcdd2c3dcce7551a404b538bd9b9c866b9b8...,0736530007 0770703004 0736531006 0706016001 07...
1,06103a5395f5579e9161899412dd594e2f3c652e0a34d3...,0399061026 0898885002 0904422001 0572998009 06...
2,060fd8c7a53060a5858e351659685bf3d518bed767cc36...,0673901005 0673901012 0706016002 0706016006 07...
3,0610702e70404ddd8efeddd130fe276dd90b0c29f203f3...,0632803014 0657852006 0657852008 0669091018 07...
4,060f99b5b9119aa44791ed09880b2868f620a3b4d302ce...,0933706001 0913688003 0896152002
